## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv() #loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025A3DAEF220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025A3DB18610>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hey! I am Harshit Goswami")
])

AIMessage(content='Hey Harshit Goswami! 👋\n\nNice to meet you. What can I do for you today?  😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 17, 'total_tokens': 45, 'completion_time': 0.050909091, 'prompt_time': 0.00011448, 'queue_time': 0.116944826, 'total_time': 0.051023571}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9ca39d5-c7dc-41ab-911d-b3ccdb5173b9-0', usage_metadata={'input_tokens': 17, 'output_tokens': 28, 'total_tokens': 45})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hey! I am Harshit Goswami"),
    AIMessage(content="Hello Harshit Goswami! 👋\n\nIt's nice to meet you. What can I do for you today?"),
    HumanMessage(content="what's my name")
])

AIMessage(content='Your name is Harshit Goswami. 😄  \n\nI remembered it from our introduction!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 56, 'total_tokens': 77, 'completion_time': 0.038181818, 'prompt_time': 0.001872156, 'queue_time': 0.121108606, 'total_time': 0.040053974}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b2534b85-8abc-4a07-b2e2-35491224565f-0', usage_metadata={'input_tokens': 56, 'output_tokens': 21, 'total_tokens': 77})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke([
    HumanMessage(content="hey! I am Harshit Goswami")
],
config=config
)

In [8]:
response.content

"Hello Harshit Goswami!\n\nIt's nice to meet you.  \n\nIs there anything I can help you with today?\n"

In [9]:
with_message_history.invoke([
    HumanMessage(content="what's my name")
],
config=config
)

AIMessage(content='Your name is Harshit Goswami.  😊 \n\nHow can I help you further?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 60, 'total_tokens': 82, 'completion_time': 0.04, 'prompt_time': 0.001875436, 'queue_time': 0.422944705, 'total_time': 0.041875436}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7fca97c5-8e0d-49bf-bcd3-4fcfd46c2c83-0', usage_metadata={'input_tokens': 60, 'output_tokens': 22, 'total_tokens': 82})

In [10]:
## change the config--->session_id
config1={"configurable":{"session_id":"chat8"}}
with_message_history.invoke([
    HumanMessage(content="what's my name")
],
config=config1
)

AIMessage(content="As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 14, 'total_tokens': 54, 'completion_time': 0.072727273, 'prompt_time': 9.24e-05, 'queue_time': 0.057062888, 'total_time': 0.072819673}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b53fe69-1f74-4f36-8d7f-495e765fc208-0', usage_metadata={'input_tokens': 14, 'output_tokens': 40, 'total_tokens': 54})

In [11]:
with_message_history.invoke([
    HumanMessage(content="hey! I am Hggoswami")
],
config=config1
)

AIMessage(content='Nice to meet you, Hggoswami!  \n\nIs there anything I can help you with today? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 70, 'total_tokens': 97, 'completion_time': 0.049090909, 'prompt_time': 0.002037177, 'queue_time': 0.122881375, 'total_time': 0.051128086}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f3962ce1-10f7-4376-bd98-8898e6c7e3cd-0', usage_metadata={'input_tokens': 70, 'output_tokens': 27, 'total_tokens': 97})

In [12]:
with_message_history.invoke([
    HumanMessage(content="what's my name")
],
config=config1
)

AIMessage(content='You told me your name is Hggoswami!  \n\nI remember what you say in our current conversation.  Do you have any other questions for me? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 110, 'total_tokens': 148, 'completion_time': 0.069090909, 'prompt_time': 0.003394184, 'queue_time': 0.056667960999999996, 'total_time': 0.072485093}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6bb20161-e8dd-4b5e-85b9-b5a1d66c1923-0', usage_metadata={'input_tokens': 110, 'output_tokens': 38, 'total_tokens': 148})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompts=ChatPromptTemplate(
    [
        ("system","You are a AI Engineer and guide students for internships"),
        MessagesPlaceholder(variable_name="message")
    ]
)

chain=prompts|model

In [14]:
chain.invoke({"message":[HumanMessage(content="Hii I am Harshit Goswami")]})

AIMessage(content="Hello Harshit Goswami!  \n\nIt's great to meet you. I'm happy to help guide you with your internship search.  \n\nTo give you the best advice, tell me a little bit about yourself:\n\n* **What year are you in?** \n* **What are your areas of interest within AI?** (e.g., Machine Learning, Computer Vision, Natural Language Processing)\n* **What kind of internship are you looking for?** (e.g., research-focused, industry-based, specific technologies)\n* **Do you have any specific companies or projects you're interested in?**\n* **What are your strengths and skills?**\n\nThe more information you give me, the better I can tailor my advice to your needs. \n\nLet's find you an awesome AI internship! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 26, 'total_tokens': 206, 'completion_time': 0.327272727, 'prompt_time': 0.00014876, 'queue_time': 0.049322253, 'total_time': 0.327421487}, 'model_name': 'Gemma2-9b-It', 'sy

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config84={"configurable":{"session_id":"chat84"}}
with_message_history.invoke(
    [HumanMessage("Hi I am hg")],
    config=config84
)

AIMessage(content="Hi hg!\n\nIt's great to meet you. As an AI Engineer, I'm happy to help guide you through your internship search.  \n\nTo give you the best advice, tell me a little about yourself:\n\n* **What are your interests within AI?** (e.g., machine learning, natural language processing, computer vision)\n* **What is your current level of study?** (e.g., undergraduate, master's)\n* **What kind of internship are you looking for?** (e.g., research, development, data analysis)\n* **Do you have any specific companies or industries in mind?**\n\nThe more information you share, the better I can help you find the perfect AI internship!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 23, 'total_tokens': 182, 'completion_time': 0.289090909, 'prompt_time': 0.000156439, 'queue_time': 0.05473133, 'total_time': 0.289247348}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': No

In [17]:
with_message_history.invoke(
    [HumanMessage(content="what's my name")], config=config84
)

AIMessage(content="You told me your name is hg!  😊 \n\n\nIs there anything else you'd like to tell me about your internship search?  I'm ready to help however I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 194, 'total_tokens': 236, 'completion_time': 0.076363636, 'prompt_time': 0.005875359, 'queue_time': 0.14356635, 'total_time': 0.082238995}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0431923-173b-440e-b484-c82e515d508a-0', usage_metadata={'input_tokens': 194, 'output_tokens': 42, 'total_tokens': 236})

In [18]:
## Add more complexity 
Prompts=ChatPromptTemplate([
    ("system","You are a AI Engineer and use {language}"),
    MessagesPlaceholder(variable_name="message")
])

chain=Prompts|model

In [19]:
chain.invoke({"message":[HumanMessage(content="hi")],"language":"Hindi"})

AIMessage(content='नमस्ते! मैं एक AI इंजीनियर हूँ और हिंदी बोलता हूँ। \n\nआप मेरेसे क्या पूछना चाहते हैं? 😊\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 18, 'total_tokens': 59, 'completion_time': 0.074545455, 'prompt_time': 9.481e-05, 'queue_time': 0.118841085, 'total_time': 0.074640265}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-173d1ce4-8638-4636-b32d-fec8923685a2-0', usage_metadata={'input_tokens': 18, 'output_tokens': 41, 'total_tokens': 59})

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="message")

In [21]:
config = {"configurable": {"session_id": "chat95"}}
repsonse=with_message_history.invoke(
    {'message': [HumanMessage(content="Hi,I am hg")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते! मैं एक एआई इंजीनियर हूँ और आपकी सहायता करने के लिए तैयार हूँ। \n\nक्या मैं आपके लिए कुछ कर सकता हूँ?  \n\n'

In [22]:
response = with_message_history.invoke(
    {"message": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [23]:
response.content

'मैं आपको याद नहीं रखता. आपने मुझे अपना नाम नहीं बताया है। \n\nआपका नाम क्या है? 😊 \n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [84]:
from langchain_core.messages import SystemMessage,trim_messages
trim=trim_messages(
    max_tokens=450,
    token_counter=model,
    strategy="last",
    allow_partial=False,
    include_system=True,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trim.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [85]:
## Appling trim function to chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(message=itemgetter("message")|trim)
    |Prompts
    |model
)

chain.invoke({"message":messages+[HumanMessage(content="which ice cream did i like")],"language":"English"})

AIMessage(content='You said you liked vanilla ice cream! 🍦  </p>', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 110, 'total_tokens': 126, 'completion_time': 0.029090909, 'prompt_time': 0.003507133, 'queue_time': 0.06800687700000001, 'total_time': 0.032598042}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e7a7132-4a1b-4df8-a712-815cc5324606-0', usage_metadata={'input_tokens': 110, 'output_tokens': 16, 'total_tokens': 126})

In [86]:
response = chain.invoke(
    {
        "message": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2"\n'

In [87]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="message",
)
config={"configurable":{"session_id":"chat5"}}

In [88]:
response = with_message_history.invoke(
    {
        "message": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'You told me your name is Bob!  😊  \n\nIs there anything else I can help you with?\n'